In [1]:
!pip install transformers
!pip install pytorch-nlp

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
# import matplotlib.pyplot as plt
import datetime
import numpy as np
import pandas as pd
import random as rn
# import sys
import time
import torch
# from keras_preprocessing.sequence import pad_sequences
# from sklearn.metrics import classification_report
# from torch import nn
from torch.optim import AdamW
# from torch.nn.utils import clip_grad_norm_
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from torchnlp.datasets import imdb_dataset  # comment out if we decide to use a downloaded dataset
from transformers import BertTokenizer, BertForSequenceClassification, BertConfig

In [3]:
# If there's a GPU available...
if torch.cuda.is_available():    

    # Tell PyTorch to use the GPU.    
    device = torch.device("cuda")

    print('There are %d GPU(s) available.' % torch.cuda.device_count())

    print('We will use the GPU:', torch.cuda.get_device_name(0))

# If not...
else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

There are 1 GPU(s) available.
We will use the GPU: Tesla T4


In [4]:
# Initializing seed values to stabilize the outcomes.
rn.seed(321)
np.random.seed(321)
torch.manual_seed(321)
torch.cuda.manual_seed(321)

In [5]:
TOKEN_MAX_LENGTH = 256
BATCH_SIZE = 8
EPOCHS = 4
LEARNING_RATE = 10e-5

## Dataset setup

In [6]:
train_data_full = imdb_dataset(train=True)
test_data_full = imdb_dataset(test=True)

rn.shuffle(train_data_full)
rn.shuffle(test_data_full)

print("\nTraining set size: {}".format(len(train_data_full)))
print("Test set size: {}".format(len(test_data_full)))


Training set size: 25000
Test set size: 25000


In [7]:
# experimenting here with a sample of dataset, to avoid memory overflow.
train_data = train_data_full[:12500]
val_data = train_data_full[12500:13000]
test_data = test_data_full

print("Training set size: {}".format(len(train_data)))
print("Test set size: {}".format(len(test_data)))
print(train_data[0])
print(test_data[0])

Training set size: 12500
Test set size: 25000
{'text': 'i\'m not even sure what to say about this film. it\'s one of only a handful of movies ever made that i would consider romantic. to try to talk plot or performance or technical details about this film would be in the words of frank zappa "like dancing about architecture". it absolutely hits the nail right on the head in the way it captures those fleeting moments in life that move us and then run away from us never to be experienced again. this seems like the movie the character version of charlie kaufman in the movie Adaptation wanted to write. the ending is left open and ambiguous, no happy ending here, just mystery. no profound life lessons, just a couple of horny and intelligent kids exploring the ability to feel the most irrational and unrealistic of feelings...... romantic love.<br /><br />10 out of 10 watch it with your special lady and recommend it to a stranger................', 'sentiment': 'pos'}
{'text': "There were heis

In [8]:
# Split data in texts and labels
train_texts, train_labels = list(zip(*map(lambda d: (d['text'], d['sentiment']), train_data)))
val_texts, val_labels = list(zip(*map(lambda d: (d['text'], d['sentiment']), val_data)))
test_texts, test_labels = list(zip(*map(lambda d: (d['text'], d['sentiment']), test_data)))

print("Train text length:  {}\tTest text length:  {}\nTrain label length: {}\tTest label length: {}".
      format(len(train_texts), len(test_texts), len(train_labels), len(test_labels)))

Train text length:  12500	Test text length:  25000
Train label length: 12500	Test label length: 25000


In [9]:
# Calculate percentage pos
train_y = np.array(train_labels) == 'pos'
val_y = np.array(val_labels) == 'pos'
test_y = np.array(test_labels) == 'pos'

print("Train records pos: {}/{}\nTest records pos: {}/{}".
      format(np.sum(train_y), len(train_y), np.sum(test_y), len(test_y)))

Train records pos: 6240/12500
Test records pos: 12500/25000


## Tokenize input and create DataLoader

In [10]:
# Token embeddings
print('Loading BERT tokenizer...')
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)

Loading BERT tokenizer...


In [11]:
# Tokenize all of the sentences and map the tokens to thier word IDs.
def tokenize_text(sentences):
  ids = []
  masks = []

  # For every sentence...
  for sent in sentences:
      # `encode_plus` will:
      #   (1) Tokenize the sentence.
      #   (2) Prepend the `[CLS]` token to the start.
      #   (3) Append the `[SEP]` token to the end.
      #   (4) Map tokens to their IDs.
      #   (5) Pad or truncate the sentence to `max_length`
      #   (6) Create attention masks for [PAD] tokens.
      encoded_dict = tokenizer.encode_plus(
                          sent,                      # Sentence to encode.
                          add_special_tokens = True, # Add '[CLS]' and '[SEP]'
                          max_length = TOKEN_MAX_LENGTH,  # Pad & truncate all sentences.
                          padding = 'max_length',
                          truncation=True,
                          return_attention_mask = True,   # Construct attn. masks.
                          return_tensors = 'pt'     # Return pytorch tensors.
                    )
      
      # Add the encoded sentence to the list.    
      ids.append(encoded_dict['input_ids'])
      
      # And its attention mask (simply differentiates padding from non-padding).
      masks.append(encoded_dict['attention_mask'])

  # Convert the lists into tensors.
  ids = torch.cat(ids, dim=0)
  masks = torch.cat(masks, dim=0)
  return ids, masks

In [12]:
train_ids, train_masks = tokenize_text(train_texts)
val_ids, val_masks = tokenize_text(val_texts)
test_ids, test_masks = tokenize_text(test_texts)

# Print sentence 0, now as a list of IDs.
print('Original: ', train_texts[0])
print('Token IDs:', train_ids[0])

Original:  i'm not even sure what to say about this film. it's one of only a handful of movies ever made that i would consider romantic. to try to talk plot or performance or technical details about this film would be in the words of frank zappa "like dancing about architecture". it absolutely hits the nail right on the head in the way it captures those fleeting moments in life that move us and then run away from us never to be experienced again. this seems like the movie the character version of charlie kaufman in the movie Adaptation wanted to write. the ending is left open and ambiguous, no happy ending here, just mystery. no profound life lessons, just a couple of horny and intelligent kids exploring the ability to feel the most irrational and unrealistic of feelings...... romantic love.<br /><br />10 out of 10 watch it with your special lady and recommend it to a stranger................
Token IDs: tensor([  101,  1045,  1005,  1049,  2025,  2130,  2469,  2054,  2000,  2360,
     

In [13]:
train_y_pos = np.array(train_labels) == 'pos'
train_y_neg = np.array(train_labels) == 'neg'
train_y_both = np.column_stack((train_y_pos, train_y_neg))
train_y_both = train_y_both.astype(float)
train_labels_tensor = torch.tensor(train_y_both)
print(train_labels_tensor.shape)

val_y_pos = np.array(val_labels) == 'pos'
val_y_neg = np.array(val_labels) == 'neg'
val_y_both = np.column_stack((val_y_pos, val_y_neg))
val_y_both = val_y_both.astype(float)
val_labels_tensor = torch.tensor(val_y_both)
print(val_labels_tensor.shape)

test_y_pos = np.array(test_labels) == 'pos'
test_y_neg = np.array(test_labels) == 'neg'
test_y_both = np.column_stack((test_y_pos, test_y_neg))
test_y_both = test_y_both.astype(float)
test_labels_tensor = torch.tensor(test_y_both)
print(test_labels_tensor.shape)

torch.Size([12500, 2])
torch.Size([500, 2])
torch.Size([25000, 2])


In [14]:
# Setting up dataloaders
train_dataset = TensorDataset(train_ids, train_masks, train_labels_tensor)
train_sampler = RandomSampler(train_dataset)
train_dataloader = DataLoader(train_dataset, sampler=train_sampler, batch_size=BATCH_SIZE)

val_dataset = TensorDataset(val_ids, val_masks, val_labels_tensor)
val_sampler = SequentialSampler(val_dataset)
val_dataloader = DataLoader(val_dataset, sampler=val_sampler, batch_size=BATCH_SIZE)

test_dataset = TensorDataset(test_ids, test_masks, test_labels_tensor)
test_sampler = SequentialSampler(test_dataset)
test_dataloader = DataLoader(test_dataset, sampler=test_sampler, batch_size=BATCH_SIZE)

## BERT

In [15]:
# Load BertForSequenceClassification, the pretrained BERT model with a single 
# linear classification layer on top. 
model = BertForSequenceClassification.from_pretrained(
    "bert-base-uncased", # Use the 12-layer BERT model, with an uncased vocab.
    num_labels = 2, # The number of output labels--2 for binary classification.
                    # You can increase this for multi-class tasks.   
    output_attentions = False, # Whether the model returns attentions weights.
    output_hidden_states = False, # Whether the model returns all hidden-states.
)

# Tell pytorch to run this model on the GPU.
model.cuda()

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, element

In [16]:
# Get all of the model's parameters as a list of tuples.
params = list(model.named_parameters())

print('The BERT model has {:} different named parameters.\n'.format(len(params)))

print('==== Embedding Layer ====\n')

for p in params[0:5]:
    print("{:<55} {:>12}".format(p[0], str(tuple(p[1].size()))))

print('\n==== First Transformer ====\n')

for p in params[5:21]:
    print("{:<55} {:>12}".format(p[0], str(tuple(p[1].size()))))

print('\n==== Output Layer ====\n')

for p in params[-4:]:
    print("{:<55} {:>12}".format(p[0], str(tuple(p[1].size()))))

The BERT model has 201 different named parameters.

==== Embedding Layer ====

bert.embeddings.word_embeddings.weight                  (30522, 768)
bert.embeddings.position_embeddings.weight                (512, 768)
bert.embeddings.token_type_embeddings.weight                (2, 768)
bert.embeddings.LayerNorm.weight                              (768,)
bert.embeddings.LayerNorm.bias                                (768,)

==== First Transformer ====

bert.encoder.layer.0.attention.self.query.weight          (768, 768)
bert.encoder.layer.0.attention.self.query.bias                (768,)
bert.encoder.layer.0.attention.self.key.weight            (768, 768)
bert.encoder.layer.0.attention.self.key.bias                  (768,)
bert.encoder.layer.0.attention.self.value.weight          (768, 768)
bert.encoder.layer.0.attention.self.value.bias                (768,)
bert.encoder.layer.0.attention.output.dense.weight        (768, 768)
bert.encoder.layer.0.attention.output.dense.bias              (

In [17]:
# Note: AdamW is a class from the huggingface library (as opposed to pytorch) 
# I believe the 'W' stands for 'Weight Decay fix"
optimizer = AdamW(model.parameters(),
                  lr = LEARNING_RATE, # args.learning_rate - default is 5e-5
                  eps = 1e-8 # args.adam_epsilon  - default is 1e-8.
                  )

In [18]:
from transformers import get_linear_schedule_with_warmup

# Total number of training steps is [number of batches] x [number of epochs]. 
# (Note that this is not the same as the number of training samples).
total_steps = len(train_dataloader) * EPOCHS

# Create the learning rate scheduler.
scheduler = get_linear_schedule_with_warmup(optimizer, 
                                            num_warmup_steps = 0, # Default value in run_glue.py
                                            num_training_steps = total_steps)

## Fine-tune BERT

### Helpers

In [48]:
# Function to calculate the accuracy of our predictions vs labels
def flat_accuracy(preds, labels):
    # pred_flat = np.argmax(preds, axis=1).flatten()
    # labels_flat = labels.flatten()
    pred_flat = np.argmax(preds, axis=1)
    labels_flat = np.argmax(labels, axis=1)
    return np.sum(pred_flat == labels_flat) / len(labels_flat)

In [20]:
def format_time(elapsed):
    '''
    Takes a time in seconds and returns a string hh:mm:ss
    '''
    # Round to the nearest second.
    elapsed_rounded = int(round((elapsed)))
    
    # Format as hh:mm:ss
    return str(datetime.timedelta(seconds=elapsed_rounded))

In [21]:
def validate_model(model, validation_dataloader):
    # Put the model in evaluation mode--the dropout layers behave differently
    # during evaluation.
    model.eval()

    # Tracking variables 
    total_eval_accuracy = 0
    total_eval_loss = 0
    nb_eval_steps = 0

    # Evaluate data for one epoch
    for batch in validation_dataloader:
        
        # Unpack this training batch from our dataloader. 
        #
        # As we unpack the batch, we'll also copy each tensor to the GPU using 
        # the `to` method.
        #
        # `batch` contains three pytorch tensors:
        #   [0]: input ids 
        #   [1]: attention masks
        #   [2]: labels 
        b_input_ids = batch[0].to(device)
        b_input_mask = batch[1].to(device)
        b_labels = batch[2].to(device)
        
        # Tell pytorch not to bother with constructing the compute graph during
        # the forward pass
        with torch.no_grad():        

            # Forward pass, calculate logit predictions.
            # token_type_ids is the same as the "segment ids", which 
            # differentiates sentence 1 and 2 in 2-sentence tasks.
            # The documentation for this `model` function is here: 
            # https://huggingface.co/transformers/v2.2.0/model_doc/bert.html#transformers.BertForSequenceClassification
            # Get the "logits" output by the model. The "logits" are the output
            # values prior to applying an activation function like the softmax.
            outputs = model(b_input_ids, 
                            token_type_ids=None, 
                            attention_mask=b_input_mask,
                            labels=b_labels.float())
            loss = outputs['loss']
            logits = outputs['logits']
            
        # Accumulate the validation loss.
        total_eval_loss += loss.item()

        # Move logits and labels to CPU
        logits = logits.detach().cpu().numpy()
        label_ids = b_labels.to('cpu').numpy()

        # Calculate the accuracy for this batch of test sentences, and
        # accumulate it over all batches.
        total_eval_accuracy += flat_accuracy(logits, label_ids)
        
    # Report the final accuracy for this validation run.
    avg_val_accuracy = total_eval_accuracy / len(validation_dataloader)

    # Calculate the average loss over all of the batches.
    avg_val_loss = total_eval_loss / len(validation_dataloader)

    return avg_val_accuracy, avg_val_loss

### Train

In [22]:
# We'll store a number of quantities such as training and validation loss, 
# validation accuracy, and timings.
training_stats = []

# Measure the total training time for the whole run.
total_t0 = time.time()

# For each epoch...
for epoch_i in range(0, EPOCHS):

    # ========================================
    #               Training
    # ========================================
    
    # Perform one full pass over the training set.

    print("")
    print('======== Epoch {:} / {:} ========'.format(epoch_i + 1, EPOCHS))
    print('Training...')

    # Measure how long the training epoch takes.
    t0 = time.time()

    # Reset the total loss for this epoch.
    total_train_loss = 0

    # Put the model into training mode. Don't be mislead--the call to 
    # `train` just changes the *mode*, it doesn't *perform* the training.
    model.train()

    # For each batch of training data...
    for step, batch in enumerate(train_dataloader):

        # Progress update every 40 batches.
        if step % 40 == 0 and not step == 0:
            # Calculate elapsed time in minutes.
            elapsed = format_time(time.time() - t0)
            
            # Report progress.
            print('  Batch {:>5,}  of  {:>5,}.    Elapsed: {:}.'.format(step, len(train_dataloader), elapsed))
        
        # Unpack this training batch from our dataloader. 
        #
        # As we unpack the batch, we'll also copy each tensor to the GPU using the 
        # `to` method.
        #
        # `batch` contains three pytorch tensors:
        #   [0]: input ids 
        #   [1]: attention masks
        #   [2]: labels 
        b_input_ids = batch[0].to(device)
        b_input_mask = batch[1].to(device)
        b_labels = batch[2].to(device)

        # Always clear any previously calculated gradients before performing a
        # backward pass.
        model.zero_grad()

        # Perform a forward pass (evaluate the model on this training batch).
        # The documentation for this `model` function is here: 
        # https://huggingface.co/transformers/v2.2.0/model_doc/bert.html#transformers.BertForSequenceClassification
        # It returns different numbers of parameters depending on what arguments
        # arge given and what flags are set. For our useage here, it returns
        # the loss (because we provided labels) and the "logits"--the model
        # outputs prior to activation.
        outputs = model(b_input_ids,
                        token_type_ids=None, 
                        attention_mask=b_input_mask, 
                        labels=b_labels.float())
        loss = outputs['loss']
        logits = outputs['logits']

        # Accumulate the training loss over all of the batches so that we can
        # calculate the average loss at the end.
        total_train_loss += loss.item()

        # Perform a backward pass to calculate the gradients.
        loss.backward()

        # This is to help prevent the "exploding gradients" problem.
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

        # Update parameters and take a step using the computed gradient.
        optimizer.step()

        # Update the learning rate.
        scheduler.step()

    # Calculate the average loss over all of the batches.
    avg_train_loss = total_train_loss / len(train_dataloader)            

    # Measure how long this epoch took.
    training_time = format_time(time.time() - t0)

    print("")
    print("  Average training loss: {0:.2f}".format(avg_train_loss))
    print("  Training epcoh took: {:}".format(training_time))

    # ========================================
    #               Validation
    # ========================================
    # After the completion of each training epoch, measure our performance on
    # our validation set.

    print("")
    print("Running Validation...")

    t0 = time.time()

    avg_val_accuracy, avg_val_loss = validate_model(model, val_dataloader)

    # Measure how long the validation run took.
    validation_time = format_time(time.time() - t0)

    print("  Accuracy: {0:.2f}".format(avg_val_accuracy))
    print("  Validation Loss: {0:.2f}".format(avg_val_loss))
    print("  Validation took: {:}".format(validation_time))

    # Record all statistics from this epoch.
    training_stats.append(
        {
            'epoch': epoch_i + 1,
            'Training Loss': avg_train_loss,
            'Valid. Loss': avg_val_loss,
            'Valid. Accur.': avg_val_accuracy,
            'Training Time': training_time,
            'Validation Time': validation_time
        }
    )

print("")
print("Training complete!")

print("Total training took {:} (h:mm:ss)".format(format_time(time.time()-total_t0)))


======== Epoch 1 / 4 ========
Training...
  Batch    40  of  1,563.    Elapsed: 0:00:15.
  Batch    80  of  1,563.    Elapsed: 0:00:29.
  Batch   120  of  1,563.    Elapsed: 0:00:44.
  Batch   160  of  1,563.    Elapsed: 0:00:59.
  Batch   200  of  1,563.    Elapsed: 0:01:14.
  Batch   240  of  1,563.    Elapsed: 0:01:30.
  Batch   280  of  1,563.    Elapsed: 0:01:45.
  Batch   320  of  1,563.    Elapsed: 0:02:00.
  Batch   360  of  1,563.    Elapsed: 0:02:15.
  Batch   400  of  1,563.    Elapsed: 0:02:31.
  Batch   440  of  1,563.    Elapsed: 0:02:46.
  Batch   480  of  1,563.    Elapsed: 0:03:01.
  Batch   520  of  1,563.    Elapsed: 0:03:16.
  Batch   560  of  1,563.    Elapsed: 0:03:32.
  Batch   600  of  1,563.    Elapsed: 0:03:47.
  Batch   640  of  1,563.    Elapsed: 0:04:02.
  Batch   680  of  1,563.    Elapsed: 0:04:17.
  Batch   720  of  1,563.    Elapsed: 0:04:33.
  Batch   760  of  1,563.    Elapsed: 0:04:48.
  Batch   800  of  1,563.    Elapsed: 0:05:03.
  Batch   840  of

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.
  import sys


  Accuracy: 0.00
  Validation Loss: 0.48
  Validation took: 0:00:07

======== Epoch 2 / 4 ========
Training...
  Batch    40  of  1,563.    Elapsed: 0:00:15.
  Batch    80  of  1,563.    Elapsed: 0:00:30.
  Batch   120  of  1,563.    Elapsed: 0:00:46.
  Batch   160  of  1,563.    Elapsed: 0:01:01.
  Batch   200  of  1,563.    Elapsed: 0:01:16.
  Batch   240  of  1,563.    Elapsed: 0:01:31.
  Batch   280  of  1,563.    Elapsed: 0:01:47.
  Batch   320  of  1,563.    Elapsed: 0:02:02.
  Batch   360  of  1,563.    Elapsed: 0:02:17.
  Batch   400  of  1,563.    Elapsed: 0:02:32.
  Batch   440  of  1,563.    Elapsed: 0:02:48.
  Batch   480  of  1,563.    Elapsed: 0:03:03.
  Batch   520  of  1,563.    Elapsed: 0:03:18.
  Batch   560  of  1,563.    Elapsed: 0:03:33.
  Batch   600  of  1,563.    Elapsed: 0:03:48.
  Batch   640  of  1,563.    Elapsed: 0:04:04.
  Batch   680  of  1,563.    Elapsed: 0:04:19.
  Batch   720  of  1,563.    Elapsed: 0:04:34.
  Batch   760  of  1,563.    Elapsed: 0:04:

In [32]:
# Display floats with two decimal places.
pd.set_option('precision', 2)

# Create a DataFrame from our training statistics.
df_stats = pd.DataFrame(data=training_stats)

# Use the 'epoch' as the row index.
df_stats = df_stats.set_index('epoch')

# A hack to force the column headers to wrap.
#df = df.style.set_table_styles([dict(selector="th",props=[('max-width', '70px')])])

# Display the table.
df_stats

,Training Loss,Valid. Loss,Valid. Accur.,Training Time,Validation Time
epoch,,,,,
1,0.61,0.48,0.0,0:09:54,0:00:07
2,0.61,0.63,0.0,0:09:54,0:00:07
3,0.63,0.70,0.0,0:09:54,0:00:07
4,0.50,0.36,0.0,0:09:54,0:00:07


### Test on test data

In [49]:
print("")
print("Running Test Set...")

t0 = time.time()

avg_accuracy, avg_loss = validate_model(model, test_dataloader)

# Measure how long the validation run took.
test_time = format_time(time.time() - t0)

print("  Accuracy: {0:.2f}".format(avg_accuracy))
print("  Loss: {0:.2f}".format(avg_loss))
print("  Time taken: {:}".format(test_time))


Running Test Set...
  Accuracy: 0.83
  Loss: 0.45
  Time taken: 0:06:08


In [50]:
model.save_pretrained("bert_imdb")